In [ ]:
import numpy as np
import pickle

data = np.load('data/rat_q_grid_modules_1_2.npz', allow_pickle=True)

print(data)  # List all arrays in the file

print(data.keys())  # List all array names

x = data['x']  # Load the array named 'x'
y = data['y']  # Load the array named 'y'
t = data['t']  # Load the array named 't'
modules = ['spikes_mod1', 'spikes_mod2']  # List the module names
for module in modules:
    print(f"Module: {module}")
    # print(f"Shape: {data[module].shape}")  # Print the shape of each module's data

NpzFile 'data/rat_q_grid_modules_1_2.npz' with keys: spikes_mod1, spikes_mod2, x, y, t...
KeysView(NpzFile 'data/rat_q_grid_modules_1_2.npz' with keys: spikes_mod1, spikes_mod2, x, y, t...)
Module: spikes_mod1
Module: spikes_mod2


In [17]:
# print type of modules KeysView(NpzFile 'data/rat_q_grid_modules_1_2.npz' with keys: spikes_mod1, spikes_mod2, x, y, t...)

print(type(data.keys()))  # Output: <class 'numpy.lib.npyio.KeysView'>
print(list(data.keys()))  # Convert to list and print
print(type(list(data.keys())))  # Output: <class 'list'>
print(data['spikes_mod1'])  # Print the data for module 1
print(data['spikes_mod2'])  # Print the data for module 2


<class 'collections.abc.KeysView'>
['spikes_mod1', 'spikes_mod2', 'x', 'y', 't', 'z', 'azimuth']
<class 'list'>


ValueError: Object arrays cannot be loaded when allow_pickle=False